In [ ]:
import torch
from diffusers import StableDiffusionPipeline
from PIL import Image

# check for gpu availability
print("Has GPU:", torch.cuda.is_available())

load stable diffusion model

In [ ]:
def load_sd_model():
    model_id = "runwayml/stable-diffusion-v1-5"

    pipe = StableDiffusionPipeline.from_pretrained(model_id)

    if torch.cuda.is_available():
        pipe = pipe.to('cuda')
        pipe = pipe.to(torch.float16)

    return pipe

generate background image

In [ ]:
def generate_background(prompt, width, height, pipe):
    result = pipe(
        prompt=prompt,
        width=width,
        height=height,
        guidance_scale=7.5,
        num_inference_steps=25
    )

    bg = result.images[0]
    return bg

testing the script

In [ ]:
if __name__ == '__main__':
    
    pipe = load_sd_model()
    pipe.enable_attention_slicing()

In [ ]:
if __name__ == '__main__':
    
    prompt = "office with work desks"
    width = 512
    height = 512

    background = generate_background(prompt, width, height, pipe)

In [ ]:
if __name__ == '__main__':
    background